In [1]:
import scanpy as sc
import squidpy as sq

import numpy as np
import pandas as pd

In [2]:
adata = sc.read_h5ad('../datasets/slideseqv2/slideseqv2_raw.h5ad')
adata = adata[:,adata.var['highly_variable_rank']<2000].copy()
sc.pp.filter_cells(adata,min_genes=20)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata

AnnData object with n_obs × n_vars = 32838 × 2000
    obs: 'barcode', 'x', 'y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_MT', 'log1p_total_counts_MT', 'pct_counts_MT', 'n_counts', 'leiden', 'cluster', 'n_genes'
    var: 'MT', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'cluster_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'spatial_neighbors', 'umap', 'log1p'
    obsm: 'X_pca', 'X_umap', 'deconvolution_results', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances', 'spatial_connectivities', 'spatial_distances'

In [ ]:
from model import SpiceMix
from sklearn.neighbors import kneighbors_graph
import torch
K, num_pcs, n_neighbors, res_lo, res_hi = 20, 20, 10, .5, 2.
context = dict(device='cuda:6', dtype=torch.float64)
repli_list = ['slide-seqv2']

A = kneighbors_graph(adata.obsm['spatial'],n_neighbors = 10)
from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    print(i)
    obj = SpiceMix(
    K=20,
    lambda_Sigma_x_inv=1e-6, power_Sigma_x_inv=2,
    repli_list=repli_list,
    context=context,
    context_Y=context,
    )
    obj.Ys = [adata.X]
    obj.Gs = [2000]
    obj.GG = 2000
    obj.Ys = [G / obj.GG * K * Y / Y.sum(1).mean() for Y, G in zip(obj.Ys, obj.Gs)]
    obj.Ys = [torch.tensor(Y, **context).pin_memory() 
               if context['device'] != 'cpu' and context['device'] == 'cpu' else
               torch.tensor(Y, **context) for Y in obj.Ys]
    obj.Es = [np.vstack((A.tocoo().row, A.tocoo().col)).T]
    obj.Es_isempty = [True, True, True]

    obj.initialize(method='kmeans',random_state=i)
    for iiter in range(10):
        obj.estimate_weights(iiter=iiter, use_spatial=[False]*obj.num_repli)
        obj.estimate_parameters(iiter=iiter, use_spatial=[False]*obj.num_repli)
    obj.initialize_Sigma_x_inv()
    for iiter in range(1, 101):
        obj.estimate_parameters(iiter=iiter, use_spatial=[True]*obj.num_repli)
        obj.estimate_weights(iiter=iiter, use_spatial=[True]*obj.num_repli)
    adata.obsm['spicemix'+str(i)] = torch.cat(obj.Xs).cpu().numpy()
    np.save('../Benchmarking_results/slide-seqv2/'+'spicemix'+str(i)+'.npy',adata.obsm['spicemix'+str(i)])

Global seed set to 0


0


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2024-03-24 07:49:25:INFO:Updating latent states
2024-03-24 07:49:28:INFO:Updating model parameters
2024-03-24 07:49:29:INFO:Q value = nan
2024-03-24 07:49:29:INFO:Updating latent states
2024-03-24 07:49:30:INFO:Updating model parameters
2024-03-24 07:49:31:INFO:Q value = nan
2024-03-24 07:49:31:INFO:Updating latent states
2024-03-24 07:49:32:INFO:Updating model parameters
2024-03-24 07:49:33:INFO:Q value = nan
2024-03-24 07:49:33:INFO:Updating latent states
2024-03-24 07:49:33:INFO:Updating model parameters
2024-03-24 07:49:34:INFO:Q value = nan
2024-03-24 07:49:34:INFO:Updating latent states
2024-03-24 07:49:35:INFO:Updating model parameters
2024-03-24 07:49:36:INFO:Q value = nan
2024-03-24 07:49:36:INFO:Updating latent states
2024-03-24 07:49:36:INFO:Updat

2024-03-24 07:54:12:INFO:Updating model parameters
2024-03-24 07:54:17:INFO:Q value = -113209907.80191508
2024-03-24 07:54:17:INFO:Updating latent states
2024-03-24 07:54:17:INFO:Updating model parameters
2024-03-24 07:54:21:INFO:Q value = -113209720.33161151
2024-03-24 07:54:21:INFO:Updating latent states
2024-03-24 07:54:22:INFO:Updating model parameters
2024-03-24 07:54:28:INFO:Q value = -113209132.90670738
2024-03-24 07:54:28:INFO:Updating latent states
2024-03-24 07:54:28:INFO:Updating model parameters
2024-03-24 07:54:47:INFO:Q value = -113203079.09929928
2024-03-24 07:54:47:INFO:Updating latent states
2024-03-24 07:54:47:INFO:Updating model parameters
2024-03-24 07:54:51:INFO:Q value = -113201804.8621883
2024-03-24 07:54:51:INFO:Updating latent states
2024-03-24 07:54:51:INFO:Updating model parameters
2024-03-24 07:54:58:INFO:Q value = -113202042.53119983
2024-03-24 07:54:58:INFO:Updating latent states
2024-03-24 07:54:58:INFO:Updating model parameters
2024-03-24 07:55:03:INFO:Q

2024-03-24 08:02:59:INFO:Q value = -113129488.49661958
2024-03-24 08:02:59:INFO:Updating latent states
2024-03-24 08:02:59:INFO:Updating model parameters
2024-03-24 08:03:01:INFO:Q value = -113129274.34540385
2024-03-24 08:03:01:INFO:Updating latent states
2024-03-24 08:03:01:INFO:Updating model parameters
2024-03-24 08:03:09:INFO:Q value = -113129152.33591975
2024-03-24 08:03:09:INFO:Updating latent states
2024-03-24 08:03:09:INFO:Updating model parameters
2024-03-24 08:03:12:INFO:Q value = -113128937.4121356
2024-03-24 08:03:12:INFO:Updating latent states
Global seed set to 1


...

2024-03-24 08:26:29:INFO:Updating model parameters
2024-03-24 08:26:34:INFO:Q value = -113206746.20131454
2024-03-24 08:26:34:INFO:Updating latent states
2024-03-24 08:26:34:INFO:Updating model parameters
2024-03-24 08:26:41:INFO:Q value = -113205867.09123237
2024-03-24 08:26:41:INFO:Updating latent states
2024-03-24 08:26:41:INFO:Updating model parameters
2024-03-24 08:26:47:INFO:Q value = -113205506.89673115
2024-03-24 08:26:47:INFO:Updating latent states
2024-03-24 08:26:47:INFO:Updating model parameters
2024-03-24 08:26:50:INFO:Q value = -113205011.74524957
2024-03-24 08:26:50:INFO:Updating latent states
2024-03-24 08:26:50:INFO:Updating model parameters
2024-03-24 08:26:59:INFO:Q value = -113201985.58065747
2024-03-24 08:26:59:INFO:Updating latent states
2024-03-24 08:26:59:INFO:Updating model parameters
2024-03-24 08:27:04:INFO:Q value = -113202002.39192924
2024-03-24 08:27:04:INFO:Updating latent states
2024-03-24 08:27:04:INFO:Updating model parameters
2024-03-24 08:27:14:INFO:

2024-03-24 08:33:37:INFO:Q value = -113103299.94857293
2024-03-24 08:33:37:INFO:Updating latent states
2024-03-24 08:33:37:INFO:Updating model parameters
2024-03-24 08:33:39:INFO:Q value = -113103008.54341881
2024-03-24 08:33:39:INFO:Updating latent states
2024-03-24 08:33:39:INFO:Updating model parameters
2024-03-24 08:33:43:INFO:Q value = -113102722.49943793
2024-03-24 08:33:43:INFO:Updating latent states
2024-03-24 08:33:43:INFO:Updating model parameters
2024-03-24 08:33:45:INFO:Q value = -113102437.29108718
2024-03-24 08:33:45:INFO:Updating latent states
Global seed set to 3


3


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2024-03-24 08:34:44:INFO:Updating latent states
2024-03-24 08:34:46:INFO:Updating model parameters
2024-03-24 08:34:47:INFO:Q value = nan
2024-03-24 08:34:47:INFO:Updating latent states
2024-03-24 08:34:47:INFO:Updating model parameters
2024-03-24 08:34:48:INFO:Q value = nan
2024-03-24 08:34:48:INFO:Updating latent states
2024-03-24 08:34:48:INFO:Updating model parameters
2024-03-24 08:34:49:INFO:Q value = nan
2024-03-24 08:34:49:INFO:Updating latent states
2024-03-24 08:34:49:INFO:Updating model parameters
2024-03-24 08:34:49:INFO:Q value = nan
2024-03-24 08:34:49:INFO:Updating latent states
2024-03-24 08:34:49:INFO:Updating model parameters
2024-03-24 08:34:50:INFO:Q value = nan
2024-03-24 08:34:50:INFO:Updating latent states
2024-03-24 08:34:50:INFO:Updat

2024-03-24 08:39:11:INFO:Updating model parameters
